In [4]:
from utils import SystemMonitor

monitor = SystemMonitor.SystemMonitor()

Might want to drop notebook for sampling because of jupyter and kernel overhead

# **ResNet50**

In [1]:
from models import SS_ResNet50
ss_resnet50 = SS_ResNet50.SS_ResNet50()

monitor.start(interval=0.5)
ss_resnet50.predict()
monitor.stop()

monitor.compute_and_print_average()
print("\nPrediction correct: " + str(ss_resnet50.correct_result()))

2023-10-26 19:38:59.906329: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 19:39:03.730654: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 19:39:03.736876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 19:39:11.399214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TypeError: 'module' object is not callable